In [17]:
import bz2
import nltk
import numpy as np
import json
import re
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/leo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
train_file = bz2.BZ2File('../input_data/train.json.bz2')
test_file = bz2.BZ2File('../input_data/test.json.bz2')
train_file = json.load(train_file)
test_file = json.load(test_file)
print(len(train_file))
print(len(test_file))

#train_file, train_links = process_input_file(train_file, valid_links, LINK_DELIM)

612271
129619


In [55]:
def process_input_file(input_file, valid_links, link_delim):
    links = []
    for i, line in enumerate(input_file):
        sentence, link = next(iter(line.items()))
        input_file[i] = sentence.lower().strip()
        if link in valid_links:
            input_file[i] = "".join([word.title() for word in input_file[i].split()])
            links.append(link)
    return " ".join(input_file), links

def encode_labels(text_with_link_markup, links, link2idx, link_delim, default_no_link):
    encoded_labels = []
    output_sentences = []
    uppercase_regex = re.compile("[A-Z][^A-Z]*")
    link_index = 0
    for sentence in nltk.sent_tokenize(text_with_link_markup):
        encoded_label = []
        output_sentence = []
        for word in nltk.word_tokenize(sentence):
            if word[0].isupper():
                sub_links = re.findall(uppercase_regex, word)
                #sub_links = filter(None, word.split(link_delim))
                for sub_link in sub_links:
                    encoded_label.append(link2idx[links[link_index]])
                    output_sentence.append(sub_link)
                link_index += 1
            else:
                encoded_label.append(default_no_link)
                output_sentence.append(word)
        encoded_labels.append(encoded_label)
        output_sentences.append(output_sentence)
    return encoded_labels, output_sentences

In [56]:
# Preprocess data ...

LINK_DELIM = '_'
link_treshold = 1
valid_links = Counter([next(iter(sentence.values())) for sentence in train_file if next(iter(sentence.values())) is not None])
valid_links = set([link for link,frequence in valid_links.items() if frequence >= link_treshold])

train_file, train_links = process_input_file(train_file, valid_links, LINK_DELIM)
test_file, test_links = process_input_file(test_file, valid_links, LINK_DELIM)

output = train_links + ["_TEXT"]
link2idx = {l:i for i,l in enumerate(output)}
idx2link = {i:l for i,l in enumerate(output)}
default_no_link = len(output) -1

train_encoded_labels, train_sentences = encode_labels(train_file, train_links, link2idx, LINK_DELIM, default_no_link)
test_encoded_labels, test_sentences = encode_labels(test_file, test_links, link2idx, LINK_DELIM, default_no_link)

vocabulary = Counter([word for sentence in train_sentences for word in sentence])
vocabulary = ['_PAD','_UNK'] + sorted(vocabulary, key=vocabulary.get, reverse=True)

word2idx = {w:i for i,w in enumerate(vocabulary)}
idx2word = {i:w for i,w in enumerate(vocabulary)} # probably vocabulary array is enough

print(len(train_links))


338198


In [135]:
x = max([x for sentence in train_encoded_labels for x in sentence])
#print(len([train_sentence for train_sentence in train_sentences for word in train_sentence if "nuclear" in word]))
#print(x)
#print([i for i in range(len(train_sentences)) if len(train_sentences[i]) != len(train_encoded_labels[i])])

#for i, sentence in enumerate(train_sentences[11:120]):

        
for i in range(120):
    sentence = train_sentences[i+110]
    _ls = []
    for j in range(len(train_encoded_labels[i])):
        idx = train_encoded_labels[i][j]
        if idx2link[idx] != "_TEXT":
            _ls.append("({}|{})".format(sentence[j], idx2link[idx]))
    if (len(_ls) > 0):
        print(_ls)
    
    
    
    
#y = 0
#print([_x for _x in [x for x in train_sentences][y]])
#print([idx2link[_x] for _x in [x for x in train_encoded_labels][y]])
#print(idx2link[x-1])
#for sentence in train_encoded_labels[:50]:
    #print([idx2link[x] for x in sentence])

IndexError: list index out of range

In [ ]:
# Encode words as integers ...

for i, sentence in enumerate(train_sentences):
    train_sentences[i] = [word2idx[word] if word in word2idx else 1 for word in sentence]
    
for i, sentence in enumerate(test_sentences):
    test_sentences[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

In [ ]:
print([idx2word[x] for x in train_sentences[0]])
print([x for x in train_file[0]])

In [7]:
train_file = bz2.BZ2File('../input_data/train.json.bz2')
train_file = json.load(train_file)
#print(len(train_file))

link_treshold = 1
valid_links = Counter([next(iter(sentence.values())) for sentence in train_file if next(iter(sentence.values())) is not None])
valid_links = set([link for link,frequence in valid_links.items() if frequence >= link_treshold])

train_links = []
for i, line in enumerate(train_file):
    sentence, link = next(iter(line.items()))
    train_file[i] = sentence.lower().strip()
    if link is not None:
        train_file[i] = "".join([word.title() for word in train_file[i].split(' ')])
        train_links.append(link)

output = train_links + ["_TEXT"]
default_no_link = len(output) - 1

train_labels = []
train_sentences = []
link_index = 0  
for sentence in nltk.sent_tokenize(" ".join(train_file)):
    train_label = []
    train_sentence = []
    for word in nltk.word_tokenize(sentence):
        if word[0].isalpha() and word[0].isupper():
            #sub_links = filter(None, word.split(LINK_DELIM))
            sub_links = re.findall('[A-Z][^A-Z]*', word)
            #try:
            for sub_link in sub_links:
                train_label.append(link_index)
                train_sentence.append(sub_link)
            link_index += 1
            #except:
             #   print(word)
        else:
            train_label.append(default_no_link)
            train_sentence.append(word)
    train_labels.append(train_label)
    train_sentences.append(train_sentence)

In [16]:
y = 104
print(train_sentences[y])
print(train_labels[y])

['in', '1933', ',', 'Bengt', 'Strömgren', 'introduced', 'the', 'term', 'hertzsprung–russell', 'diagram', 'to', 'denote', 'a', 'luminosity-spectral', 'class', 'diagram', '.']
[338198, 338198, 338198, 122, 122, 338198, 338198, 338198, 338198, 338198, 338198, 338198, 338198, 338198, 338198, 338198, 338198]
